In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from keras.utils import np_utils
from keras.models import load_model

Using TensorFlow backend.


In [0]:
def accuracy(x_test):
  y_pred = model.predict(x_test)
  # get labels from predictions
  pred_labels = np.array([np.argmax(pred) for pred in y_pred])
  #convert ground truths to column values
  y_test = np.argmax(y_test, axis=1)
  accuracy = np.mean((y_test==pred_labels))
  return accuracy

def evaluate_error(x_test):
    pred = model.predict(x_test, batch_size = 32)
    pred = np.argmax(pred, axis=1)
    pred = np.expand_dims(pred, axis=1) # make same shape as y_test
    error = np.sum(np.not_equal(pred, y_test)) / y_test.shape[0]  
    return error
  

In [3]:
# save files in drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!rm -f inference.zip

In [0]:
!mkdir folder

In [0]:
!cp "/content/gdrive/My Drive/RBC Localization/works/inference.zip" folder

In [7]:
!ls

folder	gdrive	sample_data


In [8]:
!unzip "folder/inference.zip"

Archive:  folder/inference.zip
  inflating: inference/naive.h5      
  inflating: inference/vgg.h5        
  inflating: inference/x_test.npy    
  inflating: inference/y_test.npy    


In [10]:
!ls

folder	gdrive	inference  sample_data


In [9]:
os.listdir("inference")

['naive.h5', 'x_test.npy', 'y_test.npy', 'vgg.h5']

In [17]:
x_test = np.load("inference/x_test.npy")
y_test = np.load("inference/y_test.npy")

x_test.shape, y_test.shape

((2756, 200, 200, 3), (2756, 2))

In [0]:
#plt.imshow(x_test[9])

In [0]:
model = None
model = load_model("inference/vgg.h5")
model.summary()

NameError: ignored

In [0]:
model_2 = load_model("inference/naive.h5")
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 200, 200, 32)      9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 100, 100, 64)      18496     
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 100, 100, 64)      36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 50, 50, 128)       73856     
__________

In [0]:
score_1 = model_1.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score_1[0])
print('Test accuracy:', score_1[1])

2756/2756 [==============================] - 1198s 435ms/step
Test loss: 0.13639475404447438
Test accuracy: 0.9499274310595065


In [0]:
score_2 = model_2.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score_2[0])
print('Test accuracy:', score_2[1])

2756/2756 [==============================] - 286s 104ms/step
Test loss: 0.25343694605488476
Test accuracy: 0.9499274310595065


In [0]:
# prediction for model 1
y_pred_1 = model_1.predict(x_test, verbose=1)
y_pred_1.shape

2756/2756 [==============================] - 1195s 434ms/step


(2756, 2)

In [0]:
# prediction for model 2
y_pred_2 = model_2.predict(x_test, verbose=1)
y_pred_2.shape

2756/2756 [==============================] - 292s 106ms/step


(2756, 2)

In [0]:
y_pred_1.shape, y_pred_2.shape

((2756, 2), (2756, 2))

In [0]:
# ensemble lol
y_pred = .5 *y_pred_1 + .5 *y_pred_2
y_pred.shape

(2756, 2)

In [0]:
y_pred.shape, y_test.shape

((2756, 2), (2756, 2))

In [0]:
# get labels from predictions
pred_labels = np.array([np.argmax(pred) for pred in y_pred])
pred_labels.shape

(2756,)

In [0]:
#convert ground truths to column values
y_test = np.argmax(y_test, axis=1)
y_test.shape

(2756,)

In [0]:
y_test.shape, pred_labels.shape

((2756,), (2756,))

In [0]:
print('Accuracy by ensembling two models: ',np.mean((y_test==pred_labels)))

Accuracy by ensembling two models:  0.9582728592162555


In [0]:
# .6 .4 -->>> VGG FTW
# Accuracy by ensembling two models:  0.9589985486211902

# .5 .5
# Accuracy by ensembling two models:  0.9582728592162555

In [0]:
x = 0.9582728592162555
y = 0.9499274310595065

(x-y) * 100

0.8345428156748991

In [0]:
!mkdir folder

In [0]:
!cp "/content/gdrive/My Drive/vgg.zip" folder

In [12]:
!ls

folder	gdrive	inference  sample_data


In [13]:
!unzip "folder/vgg.zip"

Archive:  folder/vgg.zip
  inflating: vgg/pred_labels_2b_1.npy  
  inflating: vgg/pred_labels_vgg_1a.npy  
  inflating: vgg/pred_labels_vgg_1b.npy  
  inflating: vgg/pred_labels_vgg_2a.npy  
  inflating: vgg/pred_labels_vgg_2b.npy  
  inflating: vgg/pred_labels_vgg_2c.npy  


In [14]:
!ls

folder	gdrive	inference  sample_data	vgg


In [15]:
p1 = np.load("vgg/pred_labels_vgg_1a.npy")
p1.shape

(2756,)

In [0]:
p2 = np.load("vgg/pred_labels_vgg_1b.npy")
p3 = np.load("vgg/pred_labels_vgg_2a.npy")
p4 = np.load("vgg/pred_labels_vgg_2b.npy")
p5 = np.load("vgg/pred_labels_vgg_2c.npy")


In [43]:
np.mean((y_test==p5))

0.9564586357039188

In [22]:
p1_e = np_utils.to_categorical(p1,2)
p1_e.shape

(2756, 2)

In [0]:
p2_e = np_utils.to_categorical(p2,2)
p3_e = np_utils.to_categorical(p3,2)
p4_e = np_utils.to_categorical(p4,2)
p5_e = np_utils.to_categorical(p5,2)

In [24]:
p1_e.shape, p2_e.shape, p3_e.shape, p4_e.shape, p5_e.shape

((2756, 2), (2756, 2), (2756, 2), (2756, 2), (2756, 2))

In [37]:
# ensemble 
y_ens = .05 *p1_e + .05 *p2_e + .3 * p3_e + .3 * p4_e + .3 * p5_e
y_ens.shape

(2756, 2)

In [26]:
y_test.shape

(2756, 2)

In [27]:
#convert ground truths to column values
y_test = np.argmax(y_test, axis=1)
y_test.shape

(2756,)

In [28]:
# get labels from predictions
y_ens_col = np.array([np.argmax(pred) for pred in y_ens])
y_ens_col.shape

(2756,)

In [38]:

y_test.shape, y_ens_col.shape

((2756,), (2756,))

In [39]:
print('Accuracy by ensembling 5 models: ',np.mean((y_test==y_ens_col)))

Accuracy by ensembling 5 models:  0.9582728592162555


In [0]:
# Snapshot ensemble -> max of 5 CNNs 95.6

# After ensemble -> 0.2% improvement